# [Debugging](https://www.tensorflow.org/programmers_guide/debugger)

**TensorFlow debugger (tfdbg)** lets you view the internal structure and states of running TensorFlow graphs during training and inference.

TensorFlow debugger uses a curses-based text user interface. On Mac OS X, install the ncurses library:
- ```$brew install homebrew/dupes/ncurses```

Use the tfdbg **command-line interface (CLI)** to debug a frequently-encountered type of bug, numerial issues, ```nan``` and ```inf```.

The following example is for users who use the low-level Session API of TensorFlow to observe such an issue:  
- ```$python -m tensorflow.python.debug.examples.debug_mnist```

The accuracy increases slightly then gets stuck:
```
Accuracy at step 0: 0.1113
Accuracy at step 1: 0.3183
Accuracy at step 2: 0.098
Accuracy at step 3: 0.098
```

## Wrapping tensorFlow sessions with tfdbg
Wrapping the session object with a debugger wrapper to add support for tfbdg:
```python
# Let your BUILD target depend on "//tensorflow/python/debug:debug_py"
# (You don't need to worry about the BUILD dependency if you are using a pip
#  install of open-source TensorFlow.)
from tensorflow.python import debug as tf_debug
sess = tf_debug.LocalCLIDebugWrapperSession(sess)
```

The code above is already in [debug_mnist.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/debug/examples/debug_mnist.py) for practice. The wrapper provides additional features, including:
- Bringing up a CLI before and after Session.run() calls, to control the execution and inspect the graph's internal state.
- Allowing you to register special filters for tensor values, to facilitate the diagnosis of issues.

In addition, the code also have already registered a tensor filter called ```tfdbg.has_inf_or_nan``` determining if there are any ```nan``` or ```inf``` values in any intermediate tensors. [API doc](https://www.tensorflow.org/api_docs/python/tfdbg/DebugDumpDir#find) of ```DebuggDumpDir.find()``` provides additional information for writing own filters.

## Debugging model training with tfdbg
Use debug_mnist as a running example.  
Start debug wrapper session:
- ```python -m tensorflow.python.debug.examples.debug_mnist --debug```  
Prompt to be about to excute the first ```Session.run()```, with information regarding the fetched tensor and feed dictionaries.

Run command:  
- ```tfdbg> run```  
  - The run command causes tfbdg to execute until the end of the next ```Session.run()```.
  - After the run ends, tfbdg displays all the dumped tensors values in the run-end CLI.
- ```tfgdb> run -t 10```
  - use the -t flag to move ahead a number of ```Session.run()``` calls at a time
  
### tfdbg CLI frequently-used commands
- ```lt``` - list dumped tensors
- ```pt <tensor>``` - print value of a dumped tensor
- ```pf <feed_tensor>``` - print a value in the feed_dict to session.run
- ```eval``` - evaluate arbitrary python and numpy expression

### Finding ```nan```'s and ```inf```'s
let the debugger repeatedly execute ```Session.run()``` calls without stopping at the run-start or run-end prompt, until the first nan or inf value shows up in the graph.
- ```tfbdg> run -f has_inf_or_nan```